In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

import xgboost as xgb
import optuna

import matplotlib.pyplot as plt
import seaborn as sns

import mlflow
import mlflow.sklearn

pd.set_option("display.max_columns", None)
sns.set_style("whitegrid")

tracking_uri = "../logs/mlruns"
os.makedirs(os.path.join(tracking_uri, ".trash"), exist_ok=True)

mlflow.set_tracking_uri(tracking_uri)
mlflow.set_experiment("house_price_prediction")


In [ ]:
import sys
import os
from pathlib import Path
import yaml


# Adjust the path to your project root folder
project_root = os.path.abspath(
    os.path.join("..")
)  # from notebooks/ up one level

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.data_loading.data_loading.data_loader import load_data_from_json
from src.data_loading.preprocessing.preprocessing import preprocess_df
from src.data_loading.preprocessing.imputation import impute_missing_values


# go two levels up from notebook dir -> project root
ROOT = (
    Path(__file__).resolve().parents[2]
    if "__file__" in globals()
    else Path.cwd().parents[1]
)
CONFIG_PATH = (
    ROOT
    / "house_price_prediction_project"
    / "config"
    / "preprocessing_config.yaml"
)

with open(CONFIG_PATH) as f:
    CONFIG = yaml.safe_load(f)

df_raw = load_data_from_json("../data/parsed_json/*.json")
df_clean = preprocess_df(
    df_raw,
    drop_raw=CONFIG["preprocessing"]["drop_raw"],
    numeric_cols=CONFIG["preprocessing"]["numeric_cols"],
)
df_clean = impute_missing_values(
    df_clean, CONFIG["preprocessing"]["imputation"]
)
# Drop price_num NaNs for the training of the model
df_clean = df_clean[df_clean["price_num"].notna()]
df_clean.drop(columns=["living_area"], inplace=True)

####

# import pandas as pd
# import numpy as np
# from sklearn.ensemble import IsolationForest

# # Copy to avoid mutating original df
# df_outlier_test = df_clean.copy()

# # Numeric features only
# numeric_cols = df_outlier_test.select_dtypes(include=['number']).columns
# X_numeric = df_outlier_test[numeric_cols]

# # 1️⃣ Detect outliers with IsolationForest
# iso = IsolationForest(
#     contamination=0.02,  # ~2% of data flagged as outliers, tune as needed
#     random_state=42
# )
# outlier_labels = iso.fit_predict(X_numeric)

# # 2️⃣ Separate outliers and inliers
# mask_outliers = outlier_labels == -1
# df_outliers = df_outlier_test[mask_outliers].reset_index(drop=True)
# df_inliers = df_outlier_test[~mask_outliers].reset_index(drop=True)

# print(f"Original shape: {df_outlier_test.shape}")
# print(f"Outliers detected: {df_outliers.shape[0]}")

# # 3️⃣ Winsorize only the upper tail (99th percentile of inliers)
# winsor_limits = {}
# df_winsorized = df_outlier_test.copy()

# for col in numeric_cols:
#     upper_limit = np.percentile(df_inliers[col], 95)
#     winsor_limits[col] = upper_limit
#     # Track which rows are capped
#     capped_mask = df_winsorized[col] > upper_limit
#     df_winsorized.loc[capped_mask, col] = upper_limit
#     if capped_mask.any():
#         print(f"{capped_mask.sum()} rows in '{col}' capped at {upper_limit:.2f}")

# print("Winsorizing applied: only upper extremes capped.")

# # ✅ Now df_winsorized can be used for modeling

# Now df_outlier_test has all rows, but extreme numeric values are capped
# df_clean = df_winsorized.copy()
####

# df_clean = df_clean[:100] 
df = df_clean.copy()

In [ ]:
from src.features.data_prep_for_modelling.data_preparation import prepare_data

FEATURES_CONFIG_PATH = (
    ROOT / "house_price_prediction_project" / "config" / "model_config.yaml"
)

# Scaled features (applies scaling according to YAML)
X_train_scaled, X_test_scaled, y_train, y_test, X_val, y_val, scaler, _ = prepare_data(
    df,
    config_path=FEATURES_CONFIG_PATH,
    model_name="linear_regression",  # uses the unified YAML key
    use_extended_features=False,       # set True if you want extended features
    cv=False
)

In [ ]:
from src.model.evaluate import ModelEvaluator
from src.model.mlflow_logger import MLFlowLogger

evaluator = ModelEvaluator()
logger = MLFlowLogger()

lr_model = LinearRegression()

# Evaluate
trained_lr, y_train_pred, y_val_pred, y_test_pred, lr_results = evaluator.evaluate(
    model=lr_model,
    X_train=X_train_scaled,
    y_train=y_train,
    X_test=X_test_scaled,
    y_test=y_test,
    model_params={},   
    fit_params={},     
    use_xgb_train=False
)

# Log the model and results
logger.log_model(trained_lr, "LinearRegression", lr_results)

In [ ]:
from src.features.feature_engineering.encoding import encode_energy_label

X_train, X_test, y_train, y_test, scaler, X_val, y_val, _ = prepare_data(
    df_clean,
    config_path=FEATURES_CONFIG_PATH, 
    model_name="random_forest",
    use_extended_features=False,     
    cv=False 
)

In [ ]:
rf_model = RandomForestRegressor()

trained_rf, y_train_pred, y_val_pred, y_test_pred, rf_results = evaluator.evaluate(
    model=rf_model,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    model_params={},  
    fit_params={},    
    use_xgb_train=False
)
logger.log_model(trained_rf, "RandomForestRegression", rf_results)

In [ ]:
from src.model.utils import load_model_config_and_search_space

X_train, X_test, y_train, y_test, X_val, y_val, scaler, _ = prepare_data(
    df_clean, config_path=FEATURES_CONFIG_PATH, model_name="xgboost", 
    use_extended_features=False, cv=False
)

MODEL_CONFIG_PATH = (
    ROOT / "house_price_prediction_project" / "config" / "model_config.yaml"
)

model_params, fit_params, _ = load_model_config_and_search_space(
    MODEL_CONFIG_PATH, model_name="xgboost"
)
fit_params_safe = fit_params.copy()
n_estimators = fit_params_safe.pop("n_estimators", 100)  

xgb_model = xgb.XGBRegressor(
    n_estimators=n_estimators,
    **model_params
)

trained_xgb, y_train_pred, y_val_pred, y_test_pred, xgb_results = evaluator.evaluate(
    xgb_model,
    X_train,
    y_train,
    X_test=X_test,
    y_test=y_test,
    fit_params=fit_params_safe, 
    use_xgb_train=False,
    X_val=X_val,
    y_val=y_val,
)
logger.log_model(trained_xgb, "XGBoostRegression", xgb_results)

In [ ]:
# df_removed

In [ ]:
X_train, X_test, y_train, y_test, X_val, y_val, scaler, _ = prepare_data(
    df_clean,
    config_path=FEATURES_CONFIG_PATH,
    model_name="xgboost_early_stopping",
    use_extended_features=False,
    cv=False,
)

xgb_model_params, xgb_fit_params, _ = load_model_config_and_search_space(
    MODEL_CONFIG_PATH, "xgboost_early_stopping"
)

xgb_model = xgb.XGBRegressor(**xgb_model_params)


trained_xgb, y_train_pred, y_val_pred, y_test_pred, xgb_results = evaluator.evaluate(
    None,
    X_train,
    y_train,
    X_test,
    y_test,
    X_val=X_val,
    y_val=y_val,
    fit_params=xgb_fit_params,
    model_params=xgb_model_params,
    use_xgb_train=True,  
)

logger.log_model(
    trained_xgb, "xgb_with_early_stopping", xgb_results, use_xgb_train=True
)

In [ ]:
# Initialize evaluator with log-transform if used
evaluator = ModelEvaluator(target_transform=np.log1p, inverse_transform=np.expm1)
xgb_model = xgb.XGBRegressor(**xgb_model_params)


trained_xgb, y_train_pred, y_val_pred, y_test_pred, xgb_results = evaluator.evaluate(
    None,
    X_train,
    y_train,
    X_test,
    y_test,
    X_val=X_val,
    y_val=y_val,
    fit_params=xgb_fit_params,
    model_params=xgb_model_params,
    use_xgb_train=True,  
)

logger.log_model(
    trained_xgb, "xgb_with_early_stopping", xgb_results, use_xgb_train=True
)


In [ ]:
####
# Outlier removal
import pandas as pd
from sklearn.ensemble import IsolationForest

# Copy to avoid mutating original df
df_outlier_test = df_clean.copy()

# 1️⃣ Select only numeric features for outlier detection
numeric_cols = df_outlier_test.select_dtypes(include=['number']).columns
X_numeric = df_outlier_test[numeric_cols]

# 2️⃣ Fit IsolationForest on numeric subset
iso = IsolationForest(
    contamination=0.02,  # ~2% flagged as outliers (tune this!)
    random_state=42
)
outlier_labels = iso.fit_predict(X_numeric)

# 3️⃣ Filter outliers (label = -1 are outliers)
df_no_outliers = df_outlier_test[outlier_labels == 1].reset_index(drop=True)

print(f"Original shape: {df_outlier_test.shape}")
print(f"After removing outliers: {df_no_outliers.shape}")

# Rows removed (outliers)
mask_outliers = outlier_labels == -1
df_removed = df_outlier_test[mask_outliers].reset_index(drop=True)

df_clean_no_outliers = df_no_outliers.copy()

In [ ]:
X_train, X_test, y_train, y_test, X_val, y_val, scaler, _ = prepare_data(
    df_clean_no_outliers,
    config_path=FEATURES_CONFIG_PATH,
    model_name="xgboost_early_stopping",
    use_extended_features=False,
    cv=False,
)

xgb_model_params, xgb_fit_params, _ = load_model_config_and_search_space(
    MODEL_CONFIG_PATH, "xgboost_early_stopping"
)

xgb_model = xgb.XGBRegressor(**xgb_model_params)


trained_xgb, y_train_pred, y_val_pred, y_test_pred, xgb_results = evaluator.evaluate(
    None,
    X_train,
    y_train,
    X_test,
    y_test,
    X_val=X_val,
    y_val=y_val,
    fit_params=xgb_fit_params,
    model_params=xgb_model_params,
    use_xgb_train=True,  
)

logger.log_model(
    trained_xgb, "xgb_with_early_stopping", xgb_results, use_xgb_train=True
)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(style="whitegrid", font_scale=1.1)

# --- Define results ---
results = {
    "Pre Outlier Removal": {
        'Test RMSE (€)': 230288.89126193404,
        'Test MAE (€)': 77554.62162063953
    },
    "Post Outlier Removal": {
        'Test RMSE (€)': 108337.01919120285,
        'Test MAE (€)': 53868.54959964413
    }
}

df_results = pd.DataFrame(results)
bar_width = 0.35
colors = ['skyblue', 'orange']

# ---------- RMSE & MAE ----------
metrics = df_results.index
y_pos = np.arange(len(metrics))

plt.figure(figsize=(10, 5))
plt.barh(y_pos - bar_width/2, df_results['Pre Outlier Removal'], height=bar_width, color=colors[0], label='Pre Outlier Removal')
plt.barh(y_pos + bar_width/2, df_results['Post Outlier Removal'], height=bar_width, color=colors[1], label='Post Outlier Removal')
plt.yticks(y_pos, metrics)
plt.gca().invert_yaxis()

# Data labels
for i, metric in enumerate(metrics):
    pre_val = df_results.loc[metric, 'Pre Outlier Removal']
    post_val = df_results.loc[metric, 'Post Outlier Removal']
    plt.text(pre_val + 5000, i - bar_width/2, f"{pre_val:,.0f}", va='center', fontsize=10)
    plt.text(post_val + 5000, i + bar_width/2, f"{post_val:,.0f}", va='center', fontsize=10)

plt.xlabel("Error (€)")
plt.title("XGBoost with Early Stopping: Test RMSE & MAE Pre vs Post Outlier Removal")
plt.legend(loc='center left', bbox_to_anchor=(1,0.5))
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(style="whitegrid", font_scale=1.1)

# --- Define results ---
results = {
    "Pre Outlier Removal": {
        'Test RMSE (€)': 230288.89,
        'Test MAE (€)': 77554.62
    },
    "Post Outlier Removal": {
        'Test RMSE (€)': 108337.02,
        'Test MAE (€)': 53868.55
    },
    "Last Week Model": {
        'Test RMSE (€)': 211172,
        'Test MAE (€)': 72964
    }
}

df_results = pd.DataFrame(results)

# Bar parameters
bar_width = 0.25
colors = ['skyblue', 'orange', 'green']

metrics = df_results.index
y_pos = np.arange(len(metrics))

plt.figure(figsize=(10, 5))

# Horizontal bars: three bars per metric
plt.barh(y_pos - bar_width, df_results['Pre Outlier Removal'], height=bar_width, color=colors[0], label='Pre Outlier Removal')
plt.barh(y_pos, df_results['Post Outlier Removal'], height=bar_width, color=colors[1], label='Post Outlier Removal')
plt.barh(y_pos + bar_width, df_results['Last Week Model'], height=bar_width, color=colors[2], label='Last Week XGB + Optuna + basic FE')

plt.yticks(y_pos, metrics)
plt.gca().invert_yaxis()

# Data labels
for i, metric in enumerate(metrics):
    for j, col in enumerate(df_results.columns):
        val = df_results.loc[metric, col]
        plt.text(val + 5000, i - bar_width + j*bar_width, f"{val:,.0f}", va='center', fontsize=10)

plt.xlabel("Error (€)")
plt.title("XGBoost Test RMSE & MAE: Pre vs Post Outlier Removal vs Last Week Model")
plt.legend(loc='center left', bbox_to_anchor=(1,0.5))
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(style="whitegrid", font_scale=1.1)

# Assume 'df_clean' has the 'price' column
prices = df_clean['price_num']

# Log-transform
log_prices = np.log1p(prices)  # log1p to handle zeros if any

plt.figure(figsize=(14, 6))

# ---------- Raw Price ----------
plt.subplot(1, 2, 1)
sns.histplot(prices, bins=60, color='skyblue', alpha=0.7)
plt.title("Raw Price Distribution")
plt.xlabel("Price (€)")
plt.ylabel("Count")
plt.grid(True, alpha=0.3)

# ---------- Log-Transformed Price ----------
plt.subplot(1, 2, 2)
sns.histplot(log_prices, bins=60, color='orange', alpha=0.7)
plt.title("Log-Transformed Price Distribution")
plt.xlabel("Log(Price + 1)")
plt.ylabel("Count")
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.ticker import FuncFormatter

sns.set(style="whitegrid", font_scale=1.1)

prices = df_clean['price_num']
log_prices = np.log1p(prices)  # log(1 + price)

# 99th percentile threshold
p99 = np.percentile(prices, 99)
extremes = prices[prices > p99]
max_val = extremes.max()
num_extremes = len(extremes)

# Formatter for raw prices
def euro_formatter(x, pos):
    return f"€{int(x/1000):,}k"  # show in thousands with k

plt.figure(figsize=(14, 6))

# ---------- Raw Price (capped at 99th percentile) ----------
plt.subplot(1, 2, 1)
sns.histplot(prices[prices <= p99], bins=60, color='skyblue', alpha=0.7)
plt.axvline(p99, color='red', linestyle='--', linewidth=2, label='99th percentile')

# Annotate extreme outliers
plt.text(p99 * 1.02, plt.gca().get_ylim()[1]*0.8, 
         f"{num_extremes} extreme listings\nup to €{int(max_val):,}", 
         color='red', fontsize=10, ha='left', va='top')

plt.title("Raw Price Distribution (capped at 99th percentile)")
plt.xlabel("Price (€)")
plt.ylabel("Count")
plt.legend()
plt.grid(True, alpha=0.3)
plt.gca().xaxis.set_major_formatter(FuncFormatter(euro_formatter))

# ---------- Log-Transformed Price ----------
plt.subplot(1, 2, 2)
sns.histplot(log_prices, bins=60, color='orange', alpha=0.7)
plt.title("Log-Transformed Price Distribution")
plt.xlabel("Log(Price + 1)")
plt.ylabel("Count")
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
